# Querying a Postgres Database via an SSH Tunnel

In [ ]:
import os
from sshtunnel import SSHTunnelForwarder
import psycopg2
import pandas as pd

In [ ]:
PORT=5432
REMOTE_SSH_PORT=22
LOCAL_BIND_PORT=55009
PRIVATE_KEY_PATH='~/.ssh/id_rsa'
TUNNEL = SSHTunnelForwarder(
        (os.getenv('SSHPROXY'), REMOTE_SSH_PORT),
        ssh_username=os.getenv('ADWUSER'),
        ssh_private_key=PRIVATE_KEY_PATH,
        remote_bind_address=(os.getenv('ADWHOST'), PORT),
        local_bind_address=('localhost', LOCAL_BIND_PORT),
        threaded=False,
)

def query(sqlQuery):
    try:
        TUNNEL.close()
        TUNNEL.start()
        with psycopg2.connect(
            database=os.getenv('ADWNAME'),
            user=os.getenv('ADWUSER'),
            password=os.getenv('ADWPSWD'),
            host=TUNNEL.local_bind_host,
            port=TUNNEL.local_bind_port,
        ) as conn:
            with conn.cursor() as cur:
                cur.execute(sqlQuery)
                temporaryTable = cur.fetchall()
                dataz = pd.DataFrame.from_records(temporaryTable, coerce_float=True)
                colnames = [desc[0] for desc in cur.description]
                columnDict = dict([(i, colnames[i]) for i in range(len(colnames))])
                dataz = dataz.rename(columns=columnDict)
                print("Query returned", "{:,}".format(len(dataz)), "row(s)")
                return dataz
    except:
        TUNNEL.close()

In [ ]:
sql = '''
YOUR QUERY GOES HERE
;'''

In [ ]:
df = query(sql)

In [ ]:
df